# 基于滑动窗口策略的机器阅读理解任务实现

也就是阅读理解的答案在文档本身里，需要定位答案在文档中的起始位置和结束位置。如果文档超过max length就截断的话问题很大，所以采用滑动窗口的思想，将其分为多个部分重叠的文档，然后在每个文档上定位答案的起始位置和结束位置，最终选择所有文档中分数最高的作为原文档中答案的起始位置和结束位置

评价指标包括精准匹配度（EM），表示预测结果与标准答案是否完全匹配，以及模糊匹配度（F1），通过字级别的P和R统计匹配程度

标签包括起始位置和具体的答案，需要进一步处理成起始token的位置和结束token的位置！有时是以列表的形式呈现，因为答案可能出现在不止一个地方或不止一个答案

最终的输出形状是[batch_size, seq_length, 2]，拆成起始位置的logits：[batch_size, seq_length]和结束位置的logits：[batch_size, seq_length]，softmax是在seq_length这个维度上进行计算的，loss是起始和结束位置的CEloss平均值！

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## Step1 导入相关包

In [2]:
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator

/home/ubuntu/anaconda3/envs/py38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Step2 数据集加载

In [3]:
# 如果可以联网，直接使用load_dataset进行加载
#datasets = load_dataset("cmrc2018", cache_dir="data")
# 如果无法联网，则使用下面的方式加载数据集
datasets = DatasetDict.load_from_disk("mrc_data")
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 10142
    })
    validation: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 3219
    })
    test: Dataset({
        features: ['id', 'context', 'question', 'answers'],
        num_rows: 1002
    })
})

In [4]:
datasets["train"][0]

{'id': 'TRAIN_186_QUERY_0',
 'context': '范廷颂枢机（，），圣名保禄·若瑟（），是越南罗马天主教枢机。1963年被任为主教；1990年被擢升为天主教河内总教区宗座署理；1994年被擢升为总主教，同年年底被擢升为枢机；2009年2月离世。范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生；童年时接受良好教育后，被一位越南神父带到河内继续其学业。范廷颂于1940年在河内大修道院完成神学学业。范廷颂于1949年6月6日在河内的主教座堂晋铎；及后被派到圣女小德兰孤儿院服务。1950年代，范廷颂在河内堂区创建移民接待中心以收容到河内避战的难民。1954年，法越战争结束，越南民主共和国建都河内，当时很多天主教神职人员逃至越南的南方，但范廷颂仍然留在河内。翌年管理圣若望小修院；惟在1960年因捍卫修院的自由、自治及拒绝政府在修院设政治课的要求而被捕。1963年4月5日，教宗任命范廷颂为天主教北宁教区主教，同年8月15日就任；其牧铭为「我信天主的爱」。由于范廷颂被越南政府软禁差不多30年，因此他无法到所属堂区进行牧灵工作而专注研读等工作。范廷颂除了面对战争、贫困、被当局迫害天主教会等问题外，也秘密恢复修院、创建女修会团体等。1990年，教宗若望保禄二世在同年6月18日擢升范廷颂为天主教河内总教区宗座署理以填补该教区总主教的空缺。1994年3月23日，范廷颂被教宗若望保禄二世擢升为天主教河内总教区总主教并兼天主教谅山教区宗座署理；同年11月26日，若望保禄二世擢升范廷颂为枢机。范廷颂在1995年至2001年期间出任天主教越南主教团主席。2003年4月26日，教宗若望保禄二世任命天主教谅山教区兼天主教高平教区吴光杰主教为天主教河内总教区署理主教；及至2005年2月19日，范廷颂因获批辞去总主教职务而荣休；吴光杰同日真除天主教河内总教区总主教职务。范廷颂于2009年2月22日清晨在河内离世，享年89岁；其葬礼于同月26日上午在天主教河内总教区总主教座堂举行。',
 'question': '范廷颂是什么时候被任为主教的？',
 'answers': {'text': ['1963年'], 'answer_start': [30]}}

## Step3 数据预处理

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/data/PLM/chinese-macbert-base")
tokenizer

BertTokenizerFast(name_or_path='/data/PLM/chinese-macbert-base', vocab_size=21128, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [6]:
sample_dataset = datasets["train"].select(range(10))

In [7]:
tokenized_examples = tokenizer(text=sample_dataset["question"],
                               text_pair=sample_dataset["context"],
                               return_offsets_mapping=True,
                               # 滑动窗口的构造通过下面两句话实现
                               return_overflowing_tokens=True, 
                               stride=128, # 两个滑动窗口之间重叠128个tokens，理解成128个token ids也行，因为是一一对应的
                               max_length=384, truncation="only_second", padding="max_length") # 填充肯定要做，截断可能是为了不让训练数据过大
tokenized_examples.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'overflow_to_sample_mapping'])

In [8]:
print(tokenized_examples["overflow_to_sample_mapping"]) # 表示属于第几个原文档
print(len(tokenized_examples["overflow_to_sample_mapping"])) # 这里就是把10个原文档用滑动窗口分成了29个文档

[0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9]
29


In [9]:
for sen in tokenizer.batch_decode(tokenized_examples["input_ids"]):
    print(sen)

[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 范 廷 颂 枢 机 （ ， ） ， 圣 名 保 禄 · 若 瑟 （ ） ， 是 越 南 罗 马 天 主 教 枢 机 。 1963 年 被 任 为 主 教 ； 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理 ； 1994 年 被 擢 升 为 总 主 教 ， 同 年 年 底 被 擢 升 为 枢 机 ； 2009 年 2 月 离 世 。 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生 ； 童 年 时 接 受 良 好 教 育 后 ， 被 一 位 越 南 神 父 带 到 河 内 继 续 其 学 业 。 范 廷 颂 于 1940 年 在 河 内 大 修 道 院 完 成 神 学 学 业 。 范 廷 颂 于 1949 年 6 月 6 日 在 河 内 的 主 教 座 堂 晋 铎 ； 及 后 被 派 到 圣 女 小 德 兰 孤 儿 院 服 务 。 1950 年 代 ， 范 廷 颂 在 河 内 堂 区 创 建 移 民 接 待 中 心 以 收 容 到 河 内 避 战 的 难 民 。 1954 年 ， 法 越 战 争 结 束 ， 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 绝 政 府 在 修 院 设 政 治 课 的 要 求 而 被 捕 。 1963 年 4 月 5 日 ， 教 宗 任 命 范 廷 颂 为 天 主 教 北 宁 教 区 主 教 ， 同 年 8 月 15 日 就 任 ； 其 牧 铭 为 「 我 信 [SEP]
[CLS] 范 廷 颂 是 什 么 时 候 被 任 为 主 教 的 ？ [SEP] 越 南 民 主 共 和 国 建 都 河 内 ， 当 时 很 多 天 主 教 神 职 人 员 逃 至 越 南 的 南 方 ， 但 范 廷 颂 仍 然 留 在 河 内 。 翌 年 管 理 圣 若 望 小 修 院 ； 惟 在 1960 年 因 捍 卫 修 院 的 自 由 、 自 治 及 拒 

In [10]:
# 即使用滑动窗口分为多个文档，offset_mapping也还是按原始文档中的位置进行计算的
print(tokenized_examples["offset_mapping"][0]) 
print(tokenized_examples["offset_mapping"][1])
print(tokenized_examples["offset_mapping"][2])
print(len(tokenized_examples["offset_mapping"][0]))
print(len(tokenized_examples["offset_mapping"][1]))
print(len(tokenized_examples["offset_mapping"][2]))

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17), (17, 18), (18, 19), (19, 20), (20, 21), (21, 22), (22, 23), (23, 24), (24, 25), (25, 26), (26, 27), (27, 28), (28, 29), (29, 30), (30, 34), (34, 35), (35, 36), (36, 37), (37, 38), (38, 39), (39, 40), (40, 41), (41, 45), (45, 46), (46, 47), (47, 48), (48, 49), (49, 50), (50, 51), (51, 52), (52, 53), (53, 54), (54, 55), (55, 56), (56, 57), (57, 58), (58, 59), (59, 60), (60, 61), (61, 62), (62, 63), (63, 67), (67, 68), (68, 69), (69, 70), (70, 71), (71, 72), (72, 73), (73, 74), (74, 75), (75, 76), (76, 77), (77, 78), (78, 79), (79, 80), (80, 81), (81, 82), (82, 83), (83, 84), (84, 85), (85, 86), (86, 87), (87, 91), (91, 92), (92, 93), (93, 94), (94, 95), (95, 96), (96, 97), (97, 98), (98, 99), (

In [11]:
print(tokenized_examples["offset_mapping"][2]) # offset_mapping显示的是token ids对应的token在原文档中的起始位置和结束位置！
# 而word ids表示token ids对应的是第几个token，
# 可以看到起始位置和结束位置基本上都相差1，这是因为中文基本上一个字就对应一个token
# 还需要注意的是tokenizer自己加的特殊字符是不算位置的（标记为(0, 0)），且阅读理解问题和文档是分别算位置的，即使它们被拼起来了
print(len(tokenized_examples["offset_mapping"][2]))

[(0, 0), (0, 1), (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (0, 0), (516, 517), (517, 518), (518, 519), (519, 520), (520, 521), (521, 522), (522, 523), (523, 524), (524, 525), (525, 526), (526, 527), (527, 528), (528, 529), (529, 530), (530, 531), (531, 532), (532, 533), (533, 534), (534, 535), (535, 536), (536, 537), (537, 538), (538, 539), (539, 540), (540, 541), (541, 542), (542, 543), (543, 544), (544, 545), (545, 546), (546, 547), (547, 548), (548, 552), (552, 553), (553, 554), (554, 555), (555, 557), (557, 558), (558, 559), (559, 560), (560, 561), (561, 562), (562, 563), (563, 564), (564, 565), (565, 566), (566, 567), (567, 568), (568, 569), (569, 570), (570, 571), (571, 572), (572, 573), (573, 574), (574, 575), (575, 576), (576, 577), (577, 578), (578, 579), (579, 580), (580, 581), (581, 582), (582, 583), (583, 584), (584, 585), (585, 586), (586, 587), (587, 588), (588, 589), (589, 590), (590, 591), 

In [12]:
sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
len(sample_mapping)

29

In [13]:
# 定位答案在token中的起始位置和结束位置
for idx, _ in enumerate(sample_mapping):
    # 先定位答案在原文档中的起始位置和结束位置。注意这段都用的是idx，没有涉及offset
    answer = sample_dataset["answers"][sample_mapping[idx]] # 注意这里不是idx而是sample_mapping[idx]，因为offset_mapping还是原文档的
    start_char = answer["answer_start"][0]
    end_char = start_char + len(answer["text"][0])
    # 下面要定位答案在各个文档token中的起始位置和结束位置，从各个文档token中context的起始和结束向答案逼近。

    # 注意这里不能用token_type_ids寻找context的起始和结束位置所对应的token ids，因为它会把特殊字符也当成0或1
    # 要用sequence_ids，其中特殊字符用None表示，第一句话用0第二句话用1标注
    # 注意这里不是sample_mapping[idx]而是idx，因为要在各个文档中都找一遍答案！
    context_start = tokenized_examples.sequence_ids(idx).index(1) # 找到第一个1，即context的起始位置对应的token
    # 找到第一个1后的第一个None，然后它的前面那个就是最后一个1，即context的结束位置对应的token
    context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

    offset = tokenized_examples.get("offset_mapping")[idx] # idx本来就应该与offset一一对应

    # 判断答案是否在context中，根据各个文档token中的起始位置和结束位置在原文档中的位置
    if offset[context_start][0] > end_char or offset[context_end][1] < start_char: # 比较的两个都是在原文档中的位置
        start_token_pos = 0
        end_token_pos = 0
    else: # 逐个查找答案在各个文档token中的起始位置和结束位置，由于截断可能也只有部分答案
        token_id = context_start
        while token_id <= context_end and offset[token_id][0] < start_char:
            token_id += 1
        start_token_pos = token_id
        token_id = context_end
        while token_id >= context_start and offset[token_id][1] > end_char:
            token_id -=1
        end_token_pos = token_id
    
    # 显然会出现offset[context_start][0]小于context_start和offset[context_end][1]大于context_end的情况，
    # 因为tokenizer可能会将多个汉字编码成一个token，却很少把一个汉字编码成多个tokens
    # 但tokenizer可能将多个英语单词编码成一个token（不太常见），也可能将一个英语单词编码成多个tokens（常见）
    # start_char, end_char, offset[context_start][0], offset[context_end][1]：原文档中的位置
    # context_start, context_end, start_token_pos, end_token_pos：token中的位置
    print(answer, start_char, end_char, context_start, context_end, offset[context_start][0], offset[context_end][1], start_token_pos, end_token_pos)
    print("token answer decode:", tokenizer.decode(tokenized_examples["input_ids"][idx][start_token_pos: end_token_pos + 1]))

{'text': ['1963年'], 'answer_start': [30]} 30 35 17 382 0 401 47 48
token answer decode: 1963 年
{'text': ['1963年'], 'answer_start': [30]} 30 35 17 382 266 659 0 0
token answer decode: [CLS]
{'text': ['1963年'], 'answer_start': [30]} 30 35 17 289 516 815 0 0
token answer decode: [CLS]
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382 0 403 53 70
token answer decode: 1990 年 被 擢 升 为 天 主 教 河 内 总 教 区 宗 座 署 理
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 382 268 664 0 0
token answer decode: [CLS]
{'text': ['1990年被擢升为天主教河内总教区宗座署理'], 'answer_start': [41]} 41 62 15 283 520 815 0 0
token answer decode: [CLS]
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382 0 403 100 124
token answer decode: 范 廷 颂 于 1919 年 6 月 15 日 在 越 南 宁 平 省 天 主 教 发 艳 教 区 出 生
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 97 126 15 382 268 664 0 0
token answer decode: [CLS]
{'text': ['范廷颂于1919年6月15日在越南宁平省天主教发艳教区出生'], 'answer_start': [97]} 

In [14]:
print(tokenized_examples.sequence_ids(idx))
print(len(tokenized_examples.sequence_ids(idx)))

[None, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, 

In [15]:
def process_func(examples): # 主要还是制作标签
    tokenized_examples = tokenizer(text=examples["question"],
                               text_pair=examples["context"],
                               return_offsets_mapping=True,
                               return_overflowing_tokens=True,
                               stride=128,
                               max_length=384, truncation="only_second", padding="max_length")
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    start_positions = []
    end_positions = []
    example_ids = []
    for idx, _ in enumerate(sample_mapping):
        # 先定位答案在原文档中的起始位置和结束位置。注意这段都用的是idx，没有涉及offset
        answer = examples["answers"][sample_mapping[idx]] # 注意这里不是idx而是sample_mapping[idx]，因为offset_mapping还是原文档的
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # 下面要定位答案在各个文档token中的起始位置和结束位置，从各个文档token中context的起始和结束向答案逼近。

        # 注意这里不能用token_type_ids寻找context的起始和结束位置所对应的token ids，因为它会把特殊字符也当成0或1
        # 要用sequence_ids，其中特殊字符用None表示，第一句话用0第二句话用1标注
        # 注意这里不是sample_mapping[idx]而是idx，因为要在各个文档中都找一遍答案！
        context_start = tokenized_examples.sequence_ids(idx).index(1) # 找到第一个1，即context的起始位置对应的token
        # 找到第一个1后的第一个None，然后它的前面那个就是最后一个1，即context的结束位置对应的token
        context_end = tokenized_examples.sequence_ids(idx).index(None, context_start) - 1

        offset = tokenized_examples.get("offset_mapping")[idx] # idx本来就应该与offset一一对应

        # 判断答案是否在context中，根据各个文档token中的起始位置和结束位置在原文档中的位置
        if offset[context_end][1] < start_char or offset[context_start][0] > end_char: # 比较的两个都是在原文档中的位置
            start_token_pos = 0
            end_token_pos = 0
        else: # 逐个查找答案在各个文档token中的起始位置和结束位置，由于截断可能也只有部分答案
            token_id = context_start
            while token_id <= context_end and offset[token_id][0] < start_char:
                token_id += 1
            start_token_pos = token_id
            token_id = context_end
            while token_id >= context_start and offset[token_id][1] > end_char:
                token_id -=1
            end_token_pos = token_id
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)
        
        example_ids.append(examples["id"][sample_mapping[idx]]) # 这是滑动窗口映射到原文档的编码！而overflow_to_sample_mapping只是映射到原文档的下标
        tokenized_examples["offset_mapping"][idx] = [
            (o if tokenized_examples.sequence_ids(idx)[k] == 1 else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][idx])
        ] # offset_mapping就不要留着特殊字符和question的部分了都设成None，只保留context；
          # 这是为了配合预测，也就是如果输出映射到offset_mapping是None那就等于没找到，可以直接判断了
        
    tokenized_examples["example_ids"] = example_ids
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    return tokenized_examples

In [16]:
tokenized_datasets = datasets.map(process_func, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 19189
    })
    validation: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 6327
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'example_ids', 'start_positions', 'end_positions'],
        num_rows: 1988
    })
})

In [17]:
print(tokenized_datasets["train"]["offset_mapping"][1])

[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, [266, 267], [267, 268], [268, 269], [269, 270], [270, 271], [271, 272], [272, 273], [273, 274], [274, 275], [275, 276], [276, 277], [277, 278], [278, 279], [279, 280], [280, 281], [281, 282], [282, 283], [283, 284], [284, 285], [285, 286], [286, 287], [287, 288], [288, 289], [289, 290], [290, 291], [291, 292], [292, 293], [293, 294], [294, 295], [295, 296], [296, 297], [297, 298], [298, 299], [299, 300], [300, 301], [301, 302], [302, 303], [303, 304], [304, 305], [305, 306], [306, 307], [307, 308], [308, 309], [309, 310], [310, 311], [311, 312], [312, 313], [313, 314], [314, 315], [315, 316], [316, 317], [317, 318], [318, 319], [319, 320], [320, 321], [321, 325], [325, 326], [326, 327], [327, 328], [328, 329], [329, 330], [330, 331], [331, 332], [332, 333], [333, 334], [334, 335], [335, 336], [336, 337], [337, 338], [338, 339], [339, 340], [340, 341], [341, 342], [342, 343], [343, 344

In [18]:
tokenized_datasets["train"]["example_ids"][:10]

['TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_0',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_1',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_2',
 'TRAIN_186_QUERY_3']

In [19]:
import collections
# 原文档的编码映射到滑动窗口
example_to_feature = collections.defaultdict(list) # 如果尝试访问defaultdict中不存在的键，defaultdict默认会自动为该键创建对应的值：空列表
for idx, example_id in enumerate(tokenized_datasets["train"]["example_ids"][:10]):
    example_to_feature[example_id].append(idx)
example_to_feature

defaultdict(list,
            {'TRAIN_186_QUERY_0': [0, 1, 2],
             'TRAIN_186_QUERY_1': [3, 4, 5],
             'TRAIN_186_QUERY_2': [6, 7, 8],
             'TRAIN_186_QUERY_3': [9]})

## Step4 获取模型输出

In [20]:
import numpy as np
import collections

# 提取最佳预测结果和相应的标签
def get_result(start_logits, end_logits, exmaples, features): 
    # start_logits和end_logits的形状都是[batch_size, seq_length]，包括特殊字符、question和context
    # exmaples表示原始数据，features就是每个滑动窗口

    predictions = {}
    references = {}

    # 原文档的编码映射到滑动窗口
    example_to_feature = collections.defaultdict(list)
    for idx, example_id in enumerate(features["example_ids"]):
        example_to_feature[example_id].append(idx)

    # 最优答案候选
    n_best = 20
    # 最大答案长度
    max_answer_length = 30

    for example in exmaples: # 对原文档逐个进行筛选
        example_id = example["id"]
        context = example["context"]
        answers = []
        for feature_idx in example_to_feature[example_id]: # 对于原文档对应的所有滑动窗口独立筛选
            start_logit = start_logits[feature_idx]
            end_logit = end_logits[feature_idx]
            offset = features[feature_idx]["offset_mapping"]
            start_indexes = np.argsort(start_logit)[::-1][:n_best].tolist() # np.argsort返回的不是值而是下标
            end_indexes = np.argsort(end_logit)[::-1][:n_best].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # 先排除不可能的答案
                    if offset[start_index] is None or offset[end_index] is None:
                        continue
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue
                    answers.append({
                        "text": context[offset[start_index][0]: offset[end_index][1]], # 原文档中的起始和结束位置
                        "score": start_logit[start_index] + end_logit[end_index] # 当前预测的分数之和
                        # 为什么不直接选start_logit最大的和end_logit最大的？其实是因为要先排除掉不可能的答案！
                    })

        if len(answers) > 0:
            # 所有滑动窗口的可行答案中取分数之和最高的！
            best_answer = max(answers, key=lambda x: x["score"])
            predictions[example_id] = best_answer["text"]
        else: # 所有滑动窗口都没有取到。。。
            predictions[example_id] = ""
        references[example_id] = example["answers"]["text"]

    return predictions, references

## Step5 评估函数

In [21]:
from cmrc_eval import evaluate_cmrc # 用了cmrc_eval.py中的evaluate_cmrc函数

def metirc(pred):
    start_logits, end_logits = pred[0]
    if start_logits.shape[0] == len(tokenized_datasets["validation"]):
        p, r = get_result(start_logits, end_logits, datasets["validation"], tokenized_datasets["validation"])
    else:
        p, r = get_result(start_logits, end_logits, datasets["test"], tokenized_datasets["test"])
    # 评价指标包括精准匹配度EM（看是否完全相等）和模糊匹配度F1（利用最长公共子串计算Precision和Recall最终合成F1）
    return evaluate_cmrc(p, r) 

## Step6 加载模型

In [22]:
model = AutoModelForQuestionAnswering.from_pretrained("/data/PLM/chinese-macbert-base")
model

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at /data/PLM/chinese-macbert-base and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elem

## Step7 配置TrainingArguments

In [23]:
args = TrainingArguments(
    output_dir="models_for_qa",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=200, # 每200步evaluate一次！
    save_strategy="epoch",
    logging_steps=50,
    num_train_epochs=3
)

## Step8 配置Trainer

In [24]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=DefaultDataCollator(), # 主要也是为了分批处理，其他上面都已经完成了，batch size见args = TrainingArguments
    compute_metrics=metirc
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


## Step9 模型训练

In [25]:
trainer.train()

Step,Training Loss,Validation Loss,Avg,F1,Em,Total,Skip
200,1.519200,1.348136,75.856872,85.575191,66.138552,3219,0
400,1.418000,1.194390,77.344989,86.407903,68.282075,3219,0
600,1.328300,1.135692,78.387408,87.529709,69.245107,3219,0
800,0.997000,1.255299,77.225157,86.789549,67.660764,3219,0
1000,1.007700,1.201607,79.023106,87.465286,70.580926,3219,0
1200,1.038300,1.206736,76.193098,85.812725,66.573470,3219,0
1400,0.773300,1.315936,77.517120,86.783231,68.251010,3219,0
1600,0.767900,1.381830,75.235335,85.326215,65.144455,3219,0
1800,0.774300,1.355522,76.278532,85.952529,66.604536,3219,0


TrainOutput(global_step=1800, training_loss=1.1303160285949707, metrics={'train_runtime': 1852.3397, 'train_samples_per_second': 31.078, 'train_steps_per_second': 0.972, 'total_flos': 1.1281552796265984e+16, 'train_loss': 1.1303160285949707, 'epoch': 3.0})

## Step8 模型预测

In [26]:
from transformers import pipeline

pipe = pipeline("question-answering", model=model, tokenizer=tokenizer, device=0)
pipe

In [27]:
pipe(question="小明在哪里上班？", context="小明在北京上班")

{'score': 0.9453522562980652, 'start': 3, 'end': 5, 'answer': '北京'}